In [3]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *

In [20]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=5, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=5, bidir=True)


        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                   
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])  

        coeff_tr = []
        biases_tr = []   

        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [5]:
data = pd.read_csv("Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv")
data = data.sample(frac=0.005, replace=True, random_state=1)
data.head()

# plot some of it
#figure(10).clear()
#plot(data[0:1000])
#title('A sample of data')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
128037,53,17,15/02/2018 11:18:07,2261,1,1,49,164,49,49,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491755,3389,6,15/02/2018 01:57:33,2837721,9,7,1144,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
470924,3389,6,15/02/2018 09:21:14,1645626,8,7,1148,1581,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
791624,3389,6,15/02/2018 02:46:24,2486564,8,8,1144,1874,677,0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,Benign
491263,53,17,15/02/2018 12:34:09,1758,1,1,30,207,30,30,...,8,0.0,0.0,0,0,0.0,0.0,0,0,Benign


In [6]:
features = ['Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg',
       'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg',
       'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg', 'Subflow Fwd Pkts',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Act Data Pkts',
       'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max',
       'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']

In [7]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [8]:
num_col = data.shape[1]
num_row = data.shape[0]

X_data = data[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = pd.DataFrame(np.nan_to_num(x_scaled))
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = data['Label'].apply(targetify)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [9]:
X.shape, y.shape

((5243, 76, 1), (5243,))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# Working with 0.01% of the data for now because my laptop sucks
#X_train2 = X_train.sample(frac=0.0005, replace=True, random_state=1)
#y_train2 = y_train.sample(frac=0.0005, replace=True, random_state=1)
#X_test2 = X_test.sample(frac=0.0005, replace=True, random_state=1)
#y_test2 = y_test.sample(frac=0.0005, replace=True, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2621, 76, 1) (2621,)
(2622, 76, 1) (2622,)


In [11]:
y_train, y_test

(285722     0
 1048343    0
 380416     0
 58913      0
 305150     0
 997654     0
 519953     0
 320091     0
 997201     0
 678468     0
 567018     0
 209673     0
 131980     0
 144499     0
 652979     0
 505006     0
 348165     0
 19275      1
 278867     0
 640329     0
 165028     0
 219165     0
 280395     0
 306676     0
 1039909    0
 438253     0
 206389     0
 311577     0
 162777     0
 967573     0
           ..
 310451     0
 250328     0
 580231     0
 527893     0
 52539      1
 1005365    0
 1043364    0
 859513     0
 563373     0
 205367     0
 266014     0
 497085     0
 978104     0
 296970     0
 830801     0
 265200     0
 1007772    0
 916666     0
 557777     0
 321709     0
 795984     0
 146309     0
 823762     0
 572926     0
 73381      0
 43650      1
 399395     0
 176520     0
 300697     0
 742140     0
 Name: Label, Length: 2621, dtype: int64, 623650     0
 662005     0
 973951     0
 623506     0
 866731     0
 54310      1
 43514      1
 910378

In [12]:
pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

In [21]:
res = Reservoir(n_internal_units=450, spectral_radius=0.6, leak=0.6,
                 connectivity=0.25, input_scaling=0.1, noise_level=0.01, circle=False)

In [ ]:
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=5, bidir=True, test = False)
input_repr

In [ ]:
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=5, bidir=True, test = True)
input_repr_te

In [ ]:
readout.fit(input_repr, y_train2)
pred_class = readout.predict(input_repr_te)
print(pred_class)
predictions = [int(round(x)) for x in pred_class]

In [ ]:
true_class = list(y_test2)
true_class

In [ ]:
df = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
df = df.sort_values('pred_class', ascending=False)
print(df.to_string())

In [ ]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)

#accuracy_score(true_class, pred_class, normalize=False)

In [157]:
#f1_score(true_class, pred_class, average='weighted')

In [19]:
(2621*810000*64)/1024/1024/1024

126.54125690460205